### Imports iniciales
Se realizan los siguientes imports:
 - arff para la apertura del archivo de datos
 - pandas para el manejo de datos en tablas
 - numpy para el manejo de operaciones matemáticas y manipulación de matrices, listas y arrays

In [1]:
!pip install qgrid
!pip install scipy
!pip install sklearn

In [2]:
import qgrid


from scipy.io import arff
import pandas as pd
import numpy as np


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.neighbors import KNeighborsClassifier

### Carga y procesamiento de los datos

In [3]:
ds = arff.loadarff('PhishingData.arff')
df = pd.DataFrame(ds[0])
df.head()
dataset = df.to_numpy()
X_parcial = dataset[: , 0:9]
Y_parcial = dataset[: , 9]
X = np.zeros(len(X_parcial))
Y = np.zeros(len(Y_parcial))
j = 0
for i in Y_parcial:
    s = i.decode('UTF-8')
    Y[j] = int(s)
    j = j + 1
X_lista = list()
for i in X_parcial:
    lista = list()
    for k in i:
        s = k.decode('UTF-8')
        lista.append(int(s))
    X_lista.append(lista)
X = np.asarray(X_lista)

### Conteo de los datos

In [4]:
muestras = len(X)
print("Hay " + str(muestras) + " muestras en el conjunto de datos.")
caracteristicas = len(X[0])
print("Hay " + str(caracteristicas) + " características en el conjunto de datos.")

Hay 1353 muestras en el conjunto de datos.
Hay 9 características en el conjunto de datos.


In [6]:
#Función para calcular error de clasificación
def error(Y_lest, Y):
    error = 0
    for ye, y in zip(Y_lest, Y):
        if ye != y:
            error += 1
    error = error/np.size(Y)
    return error


 #####################################################################################


#ETAPA 2: Ejecución de validación cruzada, clasificación k-vecinos
def k_vecinos(n_vec):
    folds = 4
    skf = StratifiedKFold(n_splits = folds)
    vec_error_clasif = np.zeros(folds)
    j = 0
    for train_index, test_index in skf.split(X , Y):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        #En un sólo pipeline se estandarizan y se procesan los datos con el clasificador_knn
        clasificador_knn = Pipeline([('scaler', StandardScaler()),('knn', KNeighborsClassifier(n_neighbors = n_vec))])
        clasificador_knn.fit(X_train , Y_train)
        Yestim = clasificador_knn.predict(X_test)
        vec_error_clasif[j] = error(Yestim, Y_test)
        j = j + 1
    return np.mean(vec_error_clasif) , np.std(vec_error_clasif)


 #####################################################################################


# Obtención tabla para determinar número óptimo de vecinos
df_types = pd.DataFrame({'Numero de vecinos' : pd.Series(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '100'])})
df_types["Error_Prueba"] = pd.Series()
df_types["Desviación estándar del error"] = ""

numero_vecinos = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 100]
for i in range(len(numero_vecinos)):
    errorprueba, desvesterror = k_vecinos(numero_vecinos[i])
    print('\nEjecución ' + str(i + 1) + ' lista.' + '\nFaltan ' + str(len(numero_vecinos) - i - 1) + ' ejecuciones.\n')
    df_types.loc[i, "Error_Prueba"] = errorprueba # cambiar por el valor correcto
    df_types.loc[i, "Desviación estándar del error"] = desvesterror  # cambiar por el valor correcto

#df_types.loc[1, "Error_Prueba"] = "0.3630" # cambiar por el valor correcto
#df_types.loc[1, "Desviación estándar del error"] = "0.0061"  # cambiar por el valor correcto
#df_types.sort_index(inplace=True)

df_types.set_index(['Numero de vecinos'], inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


TRAIN: [ 335  340  341 ... 1350 1351 1352] TEST: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 2

,Error_Prueba,Desviación estándar del error
Numero de vecinos,,
1,0.148546,0.00990912
2,0.171451,0.0186221
3,0.146325,0.0181476
4,0.158895,0.020548
5,0.142645,0.00964933
6,0.149282,0.0140693
7,0.144109,0.0125849
8,0.139684,0.00911357
9,0.143380,0.00728688
